# Chapter 7

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst's time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

If you identify a type of data manipulation that isn’t anywhere in this book or elsewhere in the pandas library, feel free to share your use case on one of the Python mailing lists or on the pandas GitHub site. Indeed, much of the design and implementation of pandas have been driven by the needs of real-world applications.

In this chapter I discuss tools for missing data, duplicate data, string manipulation, and some other analytical data transformations. In the next chapter, I focus on combining and rearranging datasets in various ways.

## 7.1 Handling Missing Data
Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is sufficient for most real-world use. For data with `float64` dtype, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data.

We call this a *sentinel value*: when present, it indicates a missing (or *null*) value:

In [2]:
import pandas as pd
import numpy as np
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [3]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

n pandas, we've adopted a convention used in the R programming language by referring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python `None` value is also treated as NA:

In [4]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [5]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [6]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [7]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

The pandas project has attempted to make working with missing data consistent across data types. Functions like `pandas.isna` abstract away many of the annoying details. See Table 7.1 for a list of some functions related to missing data handling.

**Method    |	Description**

- `dropna`	Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
- `fillna`	Fill in missing data with some value or using an interpolation method such as `"ffill"` or ``"bfill"`.
- `isna`	Return Boolean values indicating which values are missing/NA.
- `notna`	Negation of `isna`, returns `True` for non-NA values and `False` for NA values.

### Filtering Out Missing Data
There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isna and Boolean indexing, dropna can be helpful. On a Series, it returns the Series with only the nonnull data and index values:

In [8]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is the same thing as doing:


In [9]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, there are different ways to remove missing data. You may want to drop rows or columns that are all NA, or only those rows or columns containing any NAs at all. dropna by default drops any row containing a missing value:

In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how="all"` will drop only rows that are all NA:

In [12]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Keep in mind that these functions return new objects by default and do not modify the contents of the original object.

To drop columns in the same way, pass `axis="columns"`:

In [13]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [14]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the `thresh` argument:

In [15]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,-1.354613,NaN,NaN
1,-0.611999,NaN,NaN
2,-0.069826,NaN,0.648410
3,-0.304882,NaN,2.198458
4,-1.138938,0.308707,0.656997
5,-0.506726,0.036578,0.264545
6,-1.006760,-0.995034,0.687002


In [16]:
df.dropna()

,0,1,2
4,-1.138938,0.308707,0.656997
5,-0.506726,0.036578,0.264545
6,-1.006760,-0.995034,0.687002


In [17]:
df.dropna(thresh=2)

,0,1,2
2,-0.069826,NaN,0.648410
3,-0.304882,NaN,2.198458
4,-1.138938,0.308707,0.656997
5,-0.506726,0.036578,0.264545
6,-1.006760,-0.995034,0.687002


### Filling In Missing Data
Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:

In [18]:
df.fillna(0)

,0,1,2
0,-1.354613,0.000000,0.000000
1,-0.611999,0.000000,0.000000
2,-0.069826,0.000000,0.648410
3,-0.304882,0.000000,2.198458
4,-1.138938,0.308707,0.656997
5,-0.506726,0.036578,0.264545
6,-1.006760,-0.995034,0.687002


Calling `fillna` with a dictionary, you can use a different fill value for each column:

In [19]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-1.354613,0.500000,0.000000
1,-0.611999,0.500000,0.000000
2,-0.069826,0.500000,0.648410
3,-0.304882,0.500000,2.198458
4,-1.138938,0.308707,0.656997
5,-0.506726,0.036578,0.264545
6,-1.006760,-0.995034,0.687002


The same interpolation methods available for reindexing (see Table 5.3) can be used with `fillna`:

In [20]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,-1.162721,0.870135,-0.641609
1,-0.605102,-0.909846,-0.333342
2,-0.962449,NaN,0.262420
3,0.168197,NaN,-1.085810
4,-0.147476,NaN,NaN
5,-0.209359,NaN,NaN


In [21]:
df.fillna(method="ffill")

,0,1,2
0,-1.162721,0.870135,-0.641609
1,-0.605102,-0.909846,-0.333342
2,-0.962449,-0.909846,0.262420
3,0.168197,-0.909846,-1.085810
4,-0.147476,-0.909846,-1.085810
5,-0.209359,-0.909846,-1.085810


In [22]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,-1.162721,0.870135,-0.641609
1,-0.605102,-0.909846,-0.333342
2,-0.962449,-0.909846,0.262420
3,0.168197,-0.909846,-1.085810
4,-0.147476,NaN,-1.085810
5,-0.209359,NaN,-1.085810


With `fillna` you can do lots of other things such as simple data imputation using the median or mean statistics:

In [23]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

**Argument  |	Description**

- `value`	Scalar value or dictionary-like object to use to fill missing values
- `method`	Interpolation method: one of `"bfill"` (backward fill) or `"ffill"` (forward fill); default is `None`
- `axis`	Axis to fill on (`"index"` or `"columns"`); default is `axis="index"`
- `limit`	For forward and backward filling, maximum number of consecutive periods to fill

# 7.2 Data Transformation
So far in this chapter we’ve been concerned with handling missing data. Filtering, cleaning, and other transformations are another class of important operations.

## Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [24]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a Boolean Series indicating whether each row is a duplicate (its column values are exactly equal to those in an earlier row) or not:

In [25]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` returns a DataFrame with rows where the `duplicated` array is `False` filtered out:

In [26]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the `"k1"` column:

In [27]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [28]:
data.drop_duplicates(subset=['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep="last"` will return the last one:

### Transforming Data Using a Function or Mapping
For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [29]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [30]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

The `map` method on a Series (also discussed in Ch 5.2.5: Function Application and Mapping) accepts a function or dictionary-like object containing a mapping to do the transformation of values:



In [31]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:

In [32]:
def get_animal(x):
    return meat_to_animal[x]
data['food'].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using `map`is a convenient way to perform element-wise transformations and other data cleaning-related operations.

### Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value replacement. As you've already seen, `map` can be used to modify a subset of values in an object, but `replace` provides a simpler and more flexible way to do so. Let’s consider this Series:

In [33]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The `-999` values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use `replace`, producing a new Series:

In [34]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [35]:
# If you want to replace multiple values at once, you instead pass a list and then the substitute value:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [36]:
# To use a different replacement for each value, pass a list of substitutes:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [37]:
# The argument passed can also be a dictionary:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### **Note**
The `data.replace` method is distinct from `data.str.replace`, which performs element-wise string substitution. We look at these string methods on Series later in the chapter.

## Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in place without creating a new data structure. Here’s a simple example:

In [38]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])

# Like a Series, the axis indexes have a map method
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [39]:
# You can assign to the index attribute, modifying the DataFrame in place:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [40]:
# If you want to create a transformed version of a dataset without modifying the original, a useful method is rename:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [41]:
# Notably, rename can be used in conjunction with a dictionary-like object, providing new values for a subset of the axis labels:
data.rename(index={'OHIO': "INDIANA"},
            columns={'three': 'peakaboo'})

,one,two,peakaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` saves you from the chore of copying the DataFrame manually and assigning new values to its `index` and `columns` attributes.

### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [42]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use `pandas.cut`:

In [43]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. The output you see describes the bins computed by `pandas.cut`. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:

In [44]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [45]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [46]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [47]:
pd.value_counts(age_categories)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Note that `pd.value_counts(categories)` are the bin counts for the result of `pandas.cut`.

In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). You can change which side is closed by passing `right=False`:

In [48]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [49]:
# You can override the default interval-based bin labeling by passing a list or array to the labels option:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `pandas.cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [50]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.75, 1.0], (0.51, 0.75], (0.27, 0.51], (0.27, 0.51], (0.27, 0.51], ..., (0.51, 0.75], (0.026, 0.27], (0.75, 1.0], (0.51, 0.75], (0.27, 0.51]]
Length: 20
Categories (4, interval[float64, right]): [(0.026, 0.27] < (0.27, 0.51] < (0.51, 0.75] < (0.75, 1.0]]

The `precision=2` option limits the decimal precision to two digits.

A closely related function, `pandas.qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `pandas.cut` will not usually result in each bin having the same number of data points. Since `pandas.qcut` uses sample quantiles instead, you will obtain roughly equally sized bins:

In [51]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(-2.71, -0.74], (-2.71, -0.74], (-2.71, -0.74], (-2.71, -0.74], (-0.74, -0.053], ..., (0.65, 2.45], (-0.053, 0.65], (-0.74, -0.053], (0.65, 2.45], (-0.74, -0.053]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.71, -0.74] < (-0.74, -0.053] < (-0.053, 0.65] < (0.65, 2.45]]

In [52]:
pd.value_counts(quartiles)

(-2.71, -0.74]     250
(-0.74, -0.053]    250
(-0.053, 0.65]     250
(0.65, 2.45]       250
Name: count, dtype: int64

In [53]:
# Similar to pandas.cut, you can pass your own quantiles (numbers between 0 and 1, inclusive):
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-2.705, -1.275]    100
(-1.275, -0.053]    400
(-0.053, 1.25]      400
(1.25, 2.449]       100
Name: count, dtype: int64

We’ll return to `pandas.cut` and `pandas.qcut` later in the chapter during our discussion of aggregation and group operations, as these discretization functions are especially useful for quantile and group analysis.

### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [54]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.002297,-0.004067,-0.022945,0.017058
std,0.968232,0.974370,0.980563,0.993803
min,-3.131304,-2.989798,-3.296055,-2.689428
25%,-0.657505,-0.698583,-0.692703,-0.647149
50%,-0.033578,0.016821,-0.005762,0.012358
75%,0.653950,0.647581,0.676496,0.676736
max,3.050663,3.148834,3.419892,3.304693


In [55]:
# Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:
col = data[2]
col[col.abs() > 3]

309    3.095299
596   -3.296055
684   -3.061103
743    3.419892
Name: 2, dtype: float64

In [56]:
# To select all rows having a value exceeding 3 or –3, you can use the any method on a Boolean DataFrame:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
76,-2.125797,3.080869,1.066821,-1.631532
309,-0.000947,1.823691,3.095299,0.196813
393,-1.715690,3.148834,0.129177,-0.327600
397,0.927503,1.417052,-0.035580,3.304693
413,-1.732862,0.958068,-0.181204,3.234913
502,3.050663,1.318981,1.109810,-2.569380
507,-3.131304,0.249858,0.521885,-1.021996
582,0.888742,0.868193,-0.577420,3.274547
596,0.945129,1.602333,-3.296055,1.188619
684,-0.463025,-1.789697,-3.061103,1.382073


The parentheses around `data.abs() > 3` are necessary in order to call the `any` method on the result of the comparison operation.

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [57]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.002378,-0.004297,-0.023103,0.016008
std,0.967657,0.973646,0.977742,0.990503
min,-3.000000,-2.989798,-3.000000,-2.689428
25%,-0.657505,-0.698583,-0.692703,-0.647149
50%,-0.033578,0.016821,-0.005762,0.012358
75%,0.653950,0.647581,0.676496,0.676736
max,3.000000,3.000000,3.000000,3.000000


In [58]:
# The statement np.sign(data) produces 1 and –1 values based on whether the values in data are positive or negative:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,-1.0,-1.0,1.0,1.0
2,1.0,-1.0,-1.0,1.0
3,-1.0,-1.0,1.0,1.0
4,-1.0,1.0,-1.0,1.0


### Permutation and Random Sampling
Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [59]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [60]:
sampler = np.random.permutation(5)
sampler

array([4, 0, 2, 3, 1])

In [61]:
# That array can then be used in iloc-based indexing or the equivalent take function:
df.take(sampler)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
0,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13


In [62]:
df.iloc[sampler]

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
0,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13


In [63]:
# By invoking take with axis="columns", we could also select a permutation of the columns:
column_sampler = np.random.permutation(7)
column_sampler

array([1, 4, 6, 5, 0, 3, 2])

In [64]:
df.take(column_sampler, axis="columns")

,1,4,6,5,0,3,2
0,1,4,6,5,0,3,2
1,8,11,13,12,7,10,9
2,15,18,20,19,14,17,16
3,22,25,27,26,21,24,23
4,29,32,34,33,28,31,30


To select a random subset without replacement (the same row cannot appear twice), you can use the `sample` method on Series and DataFrame:

In [65]:
df.sample(n=3)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


In [66]:
# To generate a sample with replacement (to allow repeat choices), pass replace=True to sample:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

4    4
1    7
3    6
4    4
3    6
2   -1
0    5
2   -1
3    6
0    5
dtype: int64

### Computing Indicator/Dummy Variables
Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a dummy or indicator matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. pandas has a pandas.get_dummies function for doing this, though you could also devise one yourself. Let’s consider an example DataFrame:

In [67]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [68]:
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


Here I passed dtype=float to change the output type from boolean (the default in more recent versions of pandas) to floating point.

In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. pandas.get_dummies has a prefix argument for doing this:

In [69]:
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


The DataFrame.join method will be explained in more detail in the next chapter.

If a row in a DataFrame belongs to multiple categories, we have to use a different approach to create the dummy variables. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Ch 13: Data Analysis Examples:

In [70]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
                       header=None, names=mnames, engine="python")

movies[:10]

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/movielens/movies.dat'

pandas has implemented a special Series method str.get_dummies (methods that start with str. are discussed in more detail later in String Manipulation) that handles this scenario of multiple group membership encoded as a delimited string:



In [71]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

NameError: name 'movies' is not defined

Then, as before, you can combine this with movies while adding a "Genre_" to the column names in the dummies DataFrame with the add_prefix method:

In [72]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]

NameError: name 'movies' is not defined

A useful recipe for statistical applications is to combine pandas.get_dummies with a discretization function like pandas.cut:

In [73]:
np.random.seed(12345) # to make the example repeatable
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [74]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


We will look again at pandas.get_dummies later in Creating dummy variables for modeling.

# 7.3 Extension Data Types
pandas was originally built upon the capabilities present in NumPy, an array computing library used primarily for working with numerical data. Many pandas concepts, such as missing data, were implemented using what was available in NumPy while trying to maximize compatibility between libraries that used NumPy and pandas together.

Building on NumPy led to a number of shortcomings, such as:

- Missing data handling for some numerical data types, such as integers and Booleans, was incomplete. As a result, when missing data was introduced into such data, pandas converted the data type to float64 and used np.nan to represent null values. This had compounding effects by introducing subtle issues into many pandas algorithms.

- Datasets with a lot of string data were computationally expensive and used a lot of memory.

- Some data types, like time intervals, timedeltas, and timestamps with time zones, could not be supported efficiently without using computationally expensive arrays of Python objects.

More recently, pandas has developed an extension type system allowing for new data types to be added even if they are not supported natively by NumPy. These new data types can be treated as first class alongside data coming from NumPy arrays.

Let's look at an example where we create a Series of integers with a missing value:

In [75]:
s = pd.Series([1, 2, 3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

Mainly for backward compatibility reasons, Series uses the legacy behavior of using a float64 data type and np.nan for the missing value. We could create this Series instead using pandas.Int64Dtype:

In [76]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

In [77]:
s.isna()

0    False
1    False
2    False
3     True
dtype: bool

In [78]:
s.dtype

Int64Dtype()

In [79]:
# The output <NA> indicates that a value is missing for an extension type array. This uses the special pandas.NA sentinel value:
s[3]

<NA>

In [80]:
s[3] is pd.NA

True

We also could have used the shorthand "Int64" instead of pd.Int64Dtype() to specify the type. The capitalization is necessary, otherwise it will be a NumPy-based nonextension type:

In [81]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

# pandas also has an extension type specialized for string data that does not use NumPy 
# object arrays (it requires the pyarrow library, which you may need to install separately):

s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

These string arrays generally use much less memory and are frequently computationally more efficient for doing operations on large datasets.

Another important extension type is Categorical, which we discuss in more detail in Categorical Data. A reasonably complete list of extension types available as of this writing is in Table 7.3.

Extension types can be passed to the Series astype method, allowing you to convert easily as part of your data cleaning process:

In [82]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [84]:
df["A"] = df["A"].astype("Int64")
df["A"] = df["A"].astype("Int64")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,None,True


# 7.4 String Manipulation
Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

## Python Built-In String Object Methods
In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with `split`:

In [85]:
val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

In [86]:
# split is often combined with strip to trim whitespace (including line breaks):
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

In [87]:
# These substrings could be concatenated together with a two-colon delimiter using addition:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

In [89]:
# But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the join method on the string "::":
"::".join(pieces)

'a::b::guido'

In [90]:
# Other methods are concerned with locating substrings. Using Python’s in keyword is
# the best way to detect a substring, though index and find can also be used:
"guido" in val

True

In [91]:
val.index(",")

1

In [92]:
val.find(":")

-1

In [93]:
# Note that the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1):
val.index(":")

ValueError: substring not found

In [94]:
# Relatedly, count returns the number of occurrences of a particular substring:
val.count(",")

2

In [95]:
# replace will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:
val.replace(",", "::")

'a::b::  guido'

In [96]:
val.replace(",", "")

'ab  guido'

**Method   |	Description**

- `count`	Return the number of nonoverlapping occurrences of substring in the string
- `endswith`	Return `True` if string ends with suffix
- `startswith`	Return `True` if string starts with prefix
- `join`	Use string as delimiter for concatenating a sequence of other strings
- `index`	Return starting index of the first occurrence of passed substring if found in the string; otherwise, raises `ValueError` if not found
- `find`	Return position of first character of first occurrence of substring in the string; like `index`, but returns –1 if not found
- `rfind`	Return position of first character of last occurrence of substring in the string; returns –1 if not found
- `replace`	Replace occurrences of string with another string
- `strip`, `rstrip`, `lstrip`	Trim whitespace, including newlines on both sides, on the right side, or on the left side, respectively
- `split`	Break string into list of substrings using passed delimiter
- `lower`	Convert alphabet characters to lowercase
- `upper`	Convert alphabet characters to uppercase
- `casefold` Convert characters to lowercase, and convert any region-specific variable character combinations to a common comparable form
- `ljust`, `rjust`	Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width

## Regular Expressions
Regular expressions provide a flexible way to search or match (often more complex) string patterns in text. A single expression, commonly called a regex, is a string formed according to the regular expression language. Python’s built-in `re` module is responsible for applying regular expressions to strings; I’ll give a number of examples of its use here.



The `re` module functions fall into three categories: pattern matching, substitution, and splitting. Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes. Let’s look at a simple example: suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines).

The regex describing one or more whitespace characters is `\s+`:

In [97]:
import re
text = "foo    bar\t baz  \tqux"
re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

When you call `re.split(r"\s+", text)`, the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with `re.compile`, forming a reusable regex object:

In [100]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE makes the regex case insensitive

# re.IGNORECASE makes the regex case insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [101]:
# Using findall on the text produces a list of the email addresses:
In [172]: regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [102]:
In [172]: regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [103]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [104]:
text[m.start():m.end()]

'dave@google.com'

In [105]:
print(regex.match(text))

None


In [106]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


In [107]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
re.compile(pattern, flags=re.IGNORECASE)

re.compile(r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})',
           re.IGNORECASE|re.UNICODE)

In [108]:
m = regex.match("wesm@bright.net")
m.groups()

()

In [109]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [112]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

error: invalid group reference 1 at position 11

### String Functions in pandas
Cleaning up a messy dataset for analysis often requires a lot of string manipulation. To complicate matters, a column containing strings will sometimes have missing data:

In [114]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [115]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [116]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [117]:
data_as_string_ext = data.astype('string')
data_as_string_ext

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [118]:
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

In [119]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [120]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [121]:
matches.str.get(1)


Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

In [122]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [123]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


# 7.5 Categorical Data
This section introduces the pandas Categorical type. I will show how you can achieve better performance and memory use in some pandas operations by using it. I also introduce some tools that may help with using categorical data in statistics and machine learning applications.

## Background and Motivation
Frequently, a column in a table may contain repeated instances of a smaller set of distinct values. We have already seen functions like unique and value_counts, which enable us to extract the distinct values from an array and compute their frequencies, respectively:

In [124]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [125]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [126]:
pd.value_counts(values)

apple     6
orange    2
Name: count, dtype: int64

In [127]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [128]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

This representation as integers is called the categorical or dictionary-encoded representation. The array of distinct values can be called the categories, dictionary, or levels of the data. In this book we will use the terms categorical and categories. The integer values that reference the categories are called the category codes or simply codes.

The categorical representation can yield significant performance improvements when you are doing analytics. You can also perform transformations on the categories while leaving the codes unmodified. Some example transformations that can be made at relatively low cost are:

Renaming categories

Appending a new category without changing the order or position of the existing categories

## Categorical Extension Type in pandas
pandas has a special Categorical extension type for holding data that uses the integer-based categorical representation or encoding. This is a popular data compression technique for data with many occurrences of similar values and can provide significantly faster performance with lower memory use, especially for string data.

Let's consider the example Series from before:

In [129]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [130]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [131]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

In [132]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [133]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [134]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [135]:
df['fruit'] = df['fruit'].astype('category')
df["fruit"]

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [136]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [137]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [138]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [139]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

As a last note, categorical data need not be strings, even though I have shown only string examples. A categorical array can consist of any immutable value types.

### Computations with Categoricals
Using Categorical in pandas compared with the nonencoded version (like an array of strings) generally behaves the same way. Some parts of pandas, like the groupby function, perform better when working with categoricals. There are also some functions that can utilize the ordered flag.

Let's consider some random numeric data and use the pandas.qcut binning function. This returns pandas.Categorical; we used pandas.cut earlier in the book but glossed over the details of how categoricals work:

In [140]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [141]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

In [142]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [143]:
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

In [145]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


In [146]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### Better performance with categoricals
At the beginning of the section, I said that categorical types can improve performance and memory use, so let's look at some examples. Consider some Series with 10 million elements and a small number of distinct categories:

In [147]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [148]:
categories = labels.astype('category')

In [149]:
labels.memory_usage(deep=True)

600000132

In [150]:
categories.memory_usage(deep=True)

10000544

In [153]:
%time _ = labels.astype('category')

CPU times: user 407 ms, sys: 62.2 ms, total: 469 ms
Wall time: 505 ms


In [154]:
%timeit labels.value_counts()

329 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [155]:
%timeit categories.value_counts()

16.2 ms ± 94.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Categorical Methods
Series containing categorical data have several special methods similar to the Series.str specialized string methods. This also provides convenient access to the categories and codes. Consider the Series:

In [156]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [157]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [158]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [159]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [160]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [161]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

In [162]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [163]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

### Creating dummy variables for modeling
When you're using statistics or machine learning tools, you'll often transform categorical data into dummy variables, also known as one-hot encoding. This involves creating a DataFrame with a column for each distinct category; these columns contain 1s for occurrences of a given category and 0 otherwise.

Consider the previous example:

`In [264]: cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')`

As mentioned previously in this chapter, the pandas.get_dummies function converts this one-dimensional categorical data into a DataFrame containing the dummy variable:

In [164]:
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0


# 7.6 Conclusion
Effective data preparation can significantly improve productivity by enabling you to spend more time analyzing data and less time getting it ready for analysis. We have explored a number of tools in this chapter, but the coverage here is by no means comprehensive. In the next chapter, we will explore pandas's joining and grouping functionality.